In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

# Open data

In [2]:
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('credit_card_balance.csv')
previous_application = pd.read_csv('previous_application.csv')
installments_payments = pd.read_csv('installments_payments.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
# Isolate the TARGET and remove from the train dataset
y = application_train['TARGET']
del application_train['TARGET']

# Preprocess each dataset

## Train and Test

The first thing we do is preprocess the train and test dataset using the one-hot encoding for categorical values.

In [4]:
categorical_features = [col for col in application_train.columns if application_train[col].dtype == 'object']
one_hot_df = pd.concat([application_train, application_test])
one_hot_df = pd.get_dummies(one_hot_df, columns = categorical_features)

application_train = one_hot_df.iloc[:application_train.shape[0],:]
application_test = one_hot_df.iloc[application_train.shape[0]:,]

## Bureau balance

Taking a look at each dataset we see that bureau_balance is the only one lacking "CURRENT_ID" column. We need to merge it with the bureau data before merging everything in a single dataframe in order to make the predictions.

In [5]:
bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


Since it's hard to use all the informations in this dataset we will use the size, min and max of the elements in the column MONTHS_BALANCE for each SK_ID_BUREAU.

In [6]:
bureau_grouped_size = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].size()
bureau_grouped_max = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].max()
bureau_grouped_min = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min()

We do the same for the column STATUS.

In [7]:
bureau_counts = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts(normalize = False)

Count how many different values there are for each SK_ID_BUREAU

In [8]:
bureau_counts_unstacked = bureau_counts.unstack('STATUS')
bureau_counts_unstacked

STATUS,0,1,2,3,4,5,C,X
SK_ID_BUREAU,,,,,,,,
5001709,NaN,NaN,NaN,NaN,NaN,NaN,86.0,11.0
5001710,5.0,NaN,NaN,NaN,NaN,NaN,48.0,30.0
5001711,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5001712,10.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0
...,...,...,...,...,...,...,...,...
6842884,9.0,NaN,NaN,NaN,NaN,NaN,20.0,19.0
6842885,12.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN
6842886,8.0,NaN,NaN,NaN,NaN,NaN,25.0,NaN


Make a dataframe for each of the new features we got.

In [9]:
bureau_counts_unstacked.columns = ['STATUS_0', 'STATUS_1','STATUS_2','STATUS_3','STATUS_4','STATUS_5','STATUS_C','STATUS_X']

Add the MONTHS_BALANCE columns

In [10]:
bureau_counts_unstacked['MONTHS_COUNT'] = bureau_grouped_size
bureau_counts_unstacked['MONTHS_MIN'] = bureau_grouped_min
bureau_counts_unstacked['MONTHS_MAX'] = bureau_grouped_max
bureau_counts_unstacked

,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTHS_COUNT,MONTHS_MIN,MONTHS_MAX
SK_ID_BUREAU,,,,,,,,,,,
5001709,NaN,NaN,NaN,NaN,NaN,NaN,86.0,11.0,97,-96,0
5001710,5.0,NaN,NaN,NaN,NaN,NaN,48.0,30.0,83,-82,0
5001711,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4,-3,0
5001712,10.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,19,-18,0
5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,22,-21,0
...,...,...,...,...,...,...,...,...,...,...,...
6842884,9.0,NaN,NaN,NaN,NaN,NaN,20.0,19.0,48,-47,0
6842885,12.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN,24,-23,0
6842886,8.0,NaN,NaN,NaN,NaN,NaN,25.0,NaN,33,-32,0


Now that we have extracted all the information from this dataset we merge it with the bureau dataset.

In [11]:
bureau = bureau.join(bureau_counts_unstacked, how = "left", on = "SK_ID_BUREAU")
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTHS_COUNT,MONTHS_MIN,MONTHS_MAX
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Bureau

In [12]:
#One-hot encoding of categorical features as already did for the Train and Test
bureau_categorical = [bcol for bcol in bureau.columns if bureau[bcol].dtype == 'object']
bureau = pd.get_dummies(bureau, columns = bureau_categorical)

Since we have many different Bureau_ID values for each Current_ID we will use the mean of the values for the prediction.

In [13]:
average_bureau = bureau.groupby('SK_ID_CURR').mean()
# Add the parameter of number of bureau IDs for each current ID
average_bureau['bureau_count'] = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_BUREAU']
# Delete the Bureau ID since it's not an useful number
del average_bureau['SK_ID_BUREAU']
average_bureau

,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,bureau_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-735.000000,0.0,82.428571,-825.500000,NaN,0.000000,2.076236e+05,85240.928571,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
100002,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.000000,1.081319e+05,49156.200000,7997.14125,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
100003,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.000000,2.543501e+05,0.000000,202500.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
100004,-867.000000,0.0,-488.500000,-532.500000,0.000,0.000000,9.451890e+04,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
100005,-190.666667,0.0,439.333333,-123.000000,0.000,0.000000,2.190420e+05,189469.500000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-1667.076923,0.0,-1232.333333,-1364.750000,6147.000,0.000000,2.841430e+05,16307.100000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
456250,-862.000000,0.0,1288.333333,-760.000000,0.000,0.000000,1.028820e+06,744013.365000,19422.79500,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
456253,-867.500000,0.0,280.500000,-794.000000,NaN,0.000000,9.900000e+05,448958.250000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


## POS Cash Balance

Now we'll repeat the previous process for every other dataset since they all have a very similar layout.

In [14]:
POS_CASH_categorical = pd.get_dummies(POS_CASH_balance.NAME_CONTRACT_STATUS) #Only one categorical column
POS_CASH_balance.drop(columns = ["NAME_CONTRACT_STATUS"], inplace = True)
POS_CASH_balance = POS_CASH_balance.join(POS_CASH_categorical, how = "left")
POS_CASH_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,Active,Amortized debt,Approved,Canceled,Completed,Demand,Returned to the store,Signed,XNA
0,1803195,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0,0
1,1715348,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0
2,1784872,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0,0
3,1903291,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0,0
4,2341044,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,843,0,1,0,0,0,0,0,0,0,0
10001354,1717234,141565,-19,12.0,0.0,602,0,1,0,0,0,0,0,0,0,0
10001355,1283126,315695,-21,10.0,0.0,609,0,1,0,0,0,0,0,0,0,0
10001356,1082516,450255,-22,12.0,0.0,614,0,1,0,0,0,0,0,0,0,0


In [15]:
average_POS = POS_CASH_balance.groupby('SK_ID_CURR').mean()
average_POS['POS_count'] = POS_CASH_balance[['SK_ID_PREV', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_PREV']
del average_POS['SK_ID_PREV']
average_POS

,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,Active,Amortized debt,Approved,Canceled,Completed,Demand,Returned to the store,Signed,XNA,POS_count
SK_ID_CURR,,,,,,,,,,,,,,,
100001,-72.555556,4.000000,1.444444,0.777778,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,9
100002,-10.000000,24.000000,15.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,19
100003,-43.785714,10.107143,5.785714,0.000000,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,28
100004,-25.500000,3.750000,2.250000,0.000000,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,4
100005,-20.000000,11.700000,7.200000,0.000000,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-5.000000,7.875000,4.375000,0.000000,0.000000,0.777778,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0,9
456252,-79.000000,6.000000,3.000000,0.000000,0.000000,0.857143,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,7
456253,-79.235294,6.705882,2.000000,0.294118,0.294118,0.882353,0.0,0.0,0.0,0.117647,0.0,0.0,0.000000,0.0,17


## Credit card balance

In [16]:
credit_card_categorical = pd.get_dummies(credit_card_balance.NAME_CONTRACT_STATUS) #Only one categorical column
credit_card_balance.drop(columns = ["NAME_CONTRACT_STATUS"], inplace = True)
credit_card_balance = credit_card_balance.join(credit_card_categorical, how = "left")
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,Active,Approved,Completed,Demand,Refused,Sent proposal,Signed
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,35.0,0,0,1,0,0,0,0,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,69.0,0,0,1,0,0,0,0,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,30.0,0,0,1,0,0,0,0,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,10.0,0,0,1,0,0,0,0,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,101.0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.0,0,0,1,0,0,0,0,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,23.0,0,0,1,0,0,0,0,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,18.0,0,0,1,0,0,0,0,0,0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.0,0,0,1,0,0,0,0,0,0


In [17]:
average_credit_card = credit_card_balance.groupby('SK_ID_CURR').mean()
average_credit_card['credit_card_count'] = credit_card_balance[['SK_ID_PREV', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_PREV']
del average_credit_card['SK_ID_PREV']
average_credit_card

,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,...,SK_DPD,SK_DPD_DEF,Active,Approved,Completed,Demand,Refused,Sent proposal,Signed,credit_card_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,...,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,6
100011,-38.5,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.000000,3956.221849,4843.064189,4520.067568,...,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,74
100013,-48.5,18159.919219,131718.750000,6350.000000,5953.125000,0.0,0.000000,1454.539551,7168.346250,6817.172344,...,0.010417,0.010417,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,96
100021,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,...,0.000000,0.000000,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,17
100023,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,...,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456244,-21.0,131834.730732,296341.463415,24475.609756,26842.388049,0.0,2363.015854,6514.200000,32720.544878,32720.544878,...,0.000000,0.000000,0.878049,0.0,0.121951,0.0,0.0,0.0,0.0,41
456246,-5.5,13136.731875,135000.000000,0.000000,15199.256250,0.0,15199.256250,1439.150625,18778.275000,15554.340000,...,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,8
456247,-49.0,23216.396211,144000.000000,2136.315789,2149.506474,0.0,13.190684,1414.704789,4883.755263,4115.878105,...,0.031579,0.021053,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,95


## Installment payments

In the installment payments data there are no categoracal values to encode. We will proceed to make the avarage dataframe to later merge with the train e test set.

In [18]:
average_payments = installments_payments.groupby('SK_ID_CURR').mean()
average_payments['payments_count'] = installments_payments[['SK_ID_PREV', 'SK_ID_CURR']].groupby('SK_ID_CURR').count()['SK_ID_PREV']
del average_payments['SK_ID_PREV']
average_payments

,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,payments_count
SK_ID_CURR,,,,,,,
100001,1.142857,2.714286,-2187.714286,-2195.000000,5885.132143,5885.132143,7
100002,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105,19
100003,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000,25
100004,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000,3
100005,1.111111,5.000000,-586.000000,-609.555556,6240.205000,6240.205000,9
...,...,...,...,...,...,...,...
456251,1.142857,4.000000,-120.000000,-156.285714,7492.924286,7492.924286,7
456252,1.000000,3.500000,-2391.000000,-2393.833333,10069.867500,10069.867500,6
456253,1.000000,4.785714,-2372.928571,-2387.428571,4399.707857,4115.915357,14


## Previous application

In [19]:
previous_categorical = [pcol for pcol in previous_application.columns if previous_application[pcol].dtype == 'object']
previous_application = pd.get_dummies(previous_application, columns = previous_categorical)
average_previous = previous_application.groupby('SK_ID_CURR').mean()
count_previous = previous_application[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
average_previous['applications_count'] = count_previous['SK_ID_PREV']
del average_previous['SK_ID_PREV']
average_previous

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,applications_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3951.000000,24835.500,23787.00,2520.00,24835.500,13.000000,1.0,0.104326,NaN,NaN,...,0.00,0.000000,0.0,0.000000,0.0,1.00,0.0,0.0,0.0,1
100002,9251.775000,179055.000,179055.00,0.00,179055.000,9.000000,1.0,0.000000,NaN,NaN,...,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,1.0,0.0,1
100003,56553.990000,435436.500,484191.00,3442.50,435436.500,14.666667,1.0,0.050030,NaN,NaN,...,0.00,0.333333,0.0,0.333333,0.0,0.00,0.0,0.0,0.0,3
100004,5357.250000,24282.000,20106.00,4860.00,24282.000,5.000000,1.0,0.212008,NaN,NaN,...,0.00,0.000000,0.0,0.000000,0.0,0.00,1.0,0.0,0.0,1
100005,4813.200000,22308.750,20076.75,4464.00,44617.500,10.500000,1.0,0.108964,NaN,NaN,...,0.00,0.000000,0.0,0.000000,0.0,0.50,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,6605.910000,40455.000,40455.00,0.00,40455.000,17.000000,1.0,0.000000,NaN,NaN,...,0.00,0.000000,0.0,0.000000,0.0,1.00,0.0,0.0,0.0,1
456252,10074.465000,57595.500,56821.50,3456.00,57595.500,10.000000,1.0,0.062443,NaN,NaN,...,0.00,0.000000,1.0,0.000000,0.0,0.00,0.0,0.0,0.0,1
456253,4770.405000,24162.750,20625.75,4403.25,24162.750,11.500000,1.0,0.214316,NaN,NaN,...,0.00,0.000000,0.0,0.000000,0.0,1.00,0.0,0.0,0.0,2


# Join dataframes

For each dataframe we got we will merge it with the original train and test dataframes by the SK_ID_CURR.

In [20]:
# Bureau
application_train = application_train.merge(right = average_bureau.reset_index(), how = 'left', on = 'SK_ID_CURR')
application_test = application_test.merge(right = average_bureau.reset_index(), how = 'left', on = 'SK_ID_CURR')

# Credit card
application_train = application_train.merge(right = average_credit_card.reset_index(), how = 'left', on = 'SK_ID_CURR')
application_test = application_test.merge(right = average_credit_card.reset_index(), how = 'left', on = 'SK_ID_CURR')

# Payments
application_train = application_train.merge(right = average_payments.reset_index(), how = 'left', on = 'SK_ID_CURR')
application_test = application_test.merge(right = average_payments.reset_index(), how = 'left', on = 'SK_ID_CURR')

# POS
application_train = application_train.merge(right = average_POS.reset_index(), how = 'left', on = 'SK_ID_CURR')
application_test = application_test.merge(right = average_POS.reset_index(), how = 'left', on = 'SK_ID_CURR')

# Previous applications
application_train = application_train.merge(right = average_previous.reset_index(), how = 'left', on = 'SK_ID_CURR')
application_test = application_test.merge(right = average_previous.reset_index(), how = 'left', on = 'SK_ID_CURR')

# Missing values

Remove the features with more than 70% missing values

In [21]:
#for col in application_test:
#    if application_train[col].isnull().sum()/len(col) > 0.8:
#        del application_test[col]

#for col in application_train:
#    if application_train[col].isnull().sum()/len(col) > 0.8:
#        del application_train[col]

In [22]:
del application_train["SK_ID_CURR"]
del application_test["SK_ID_CURR"]

# Predictions

In [24]:
train_x, valid_x, train_y, valid_y = train_test_split(application_train, y, test_size = 0.2, shuffle = True)
train_data = lgb.Dataset(train_x, label = train_y)
valid_data = lgb.Dataset(valid_x, label = valid_y)
application_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in application_train.columns]
application_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in application_test.columns]

# Parameters
params = {'boosting_type': 'gbdt',
          'max_depth' : 10,
          'objective': 'binary',
          'nthread': 5,
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'
          }

lgbm = lgb.train(params,
                 train_data,
                 2500,
                 valid_sets = valid_data,
                 early_stopping_rounds = 100,
                 verbose_eval = 10
                 )

predictions_lgbm_prob = lgbm.predict(application_test)

submission.TARGET = predictions_lgbm_prob

submission.to_csv('submission.csv', index=False)


Training until validation scores don't improve for 100 rounds
[10]	valid_0's auc: 0.736055
[20]	valid_0's auc: 0.743482
[30]	valid_0's auc: 0.748577
[40]	valid_0's auc: 0.753356
[50]	valid_0's auc: 0.757279
[60]	valid_0's auc: 0.760491
[70]	valid_0's auc: 0.763688
[80]	valid_0's auc: 0.766118
[90]	valid_0's auc: 0.767951
[100]	valid_0's auc: 0.769183
[110]	valid_0's auc: 0.770348
[120]	valid_0's auc: 0.771107
[130]	valid_0's auc: 0.772044
[140]	valid_0's auc: 0.772688
[150]	valid_0's auc: 0.773284
[160]	valid_0's auc: 0.773744
[170]	valid_0's auc: 0.774257
[180]	valid_0's auc: 0.774672
[190]	valid_0's auc: 0.775046
[200]	valid_0's auc: 0.775522
[210]	valid_0's auc: 0.775837
[220]	valid_0's auc: 0.776098
[230]	valid_0's auc: 0.776332
[240]	valid_0's auc: 0.776589
[250]	valid_0's auc: 0.776771
[260]	valid_0's auc: 0.777005
[270]	valid_0's auc: 0.777221
[280]	valid_0's auc: 0.777416
[290]	valid_0's auc: 0.77745
[300]	valid_0's auc: 0.777529
[310]	valid_0's auc: 0.777639
[320]	valid_0's au

In [ ]:
# Define a function that returns a summary of the missing values
#def find_missing(data):
#    # number of missing values
#    count_missing = data.isnull().sum().values
#    # total records
#    total = data.shape[0]
#    # percentage of missing
#    ratio_missing = count_missing / total
#    # return a dataframe to show: feature name, # of missing and % of missing
#    return pd.DataFrame(data = {'missing_count' : count_missing, 'missing_ratio' : ratio_missing}, index = data.columns.values)